In [65]:
#files to open: utils, GRU_sequence_weights.h5, folder Search/search, output, train.csv

from google.colab import files

uploaded = files.upload()


Saving search to search


In [3]:
import os
import shutil 
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler



from utils import model_LSTM

Using TensorFlow backend.


In [0]:
def deltas_in_search(file_name = 'search'):
    file_search = open(file_name, 'r')

    count = 0
    for line in file_search:
        count +=1
        if count == 5:
            break
        continue

    deltas = {}

    for line in file_search:
        if line.find('Searching') != -1:
            J0 = int(line.split()[1])
            deltas[J0] = []
            E0 = float(line.split()[-1])
            continue

        if line.find('No') != -1 or \
           len(line.split()) == 0 or \
           int(line.split()[0]) != int(J0) - 1:
            continue

        try:
            E = float(line.split()[-1])
            delta = round(E - E0, 3)
            deltas[J0].append(delta)
        except ValueError:
            break
            
    return deltas


# Writing out predictions

In [5]:
import csv
import random
random.seed(227)

file_csv = open('output_predictions', 'w', newline='')

filewriter = csv.writer(file_csv, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
filewriter.writerow([1,2,3,4,5,6])
for i in range(int(len(data)/2), len(data)):
    filewriter.writerow(data[i])

#files.download('train.csv')


NameError: ignored

In [0]:
file_csv.close()
files.download('test.csv')


# RNN model

In [0]:
from keras.models import Model, Sequential
from keras.layers import Dense, Input, GRU, LSTM
from keras.optimizers import Adam


def model_LSTM2(hidden_size, Tx):  
    model = Sequential()
    model.add(LSTM(hidden_size, return_sequences=False, input_shape=(Tx,1)))
    model.add(Dense(units=1, activation='sigmoid'))
  
    return model


In [7]:
Tx, Ty =  6, 1

model = model_LSTM2(16, Tx)
print(model.summary())


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 16)                1152      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 1,169
Trainable params: 1,169
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#LSTM_model.load_weights('sources/LSTM_sequence_weights.h5')
model.load_weights('LSTM_sequence_weights.h5')

# To get search file use Prepare_test notebook

# Calculate deltas on base on ready search file

In [0]:
seria_name, quan_num = 'J1J-1', [1, 1, 0]
peak_file = 'expfile_st'

In [10]:
deltas = deltas_in_search()
k_ini = list(deltas.keys())[0]

deltas = {k:v for k,v in deltas.items() if k < Tx+k_ini}
print(deltas)


{2: [-0.465, -0.233, -0.159, 0.091, 0.159, 0.452], 3: [-0.49, -0.45, -0.354, -0.306, -0.281, -0.237, -0.232, -0.152, -0.113, -0.066, -0.059, 0.074, 0.201, 0.256, 0.266, 0.314, 0.398], 4: [-0.237, -0.226, -0.217, -0.191, -0.15, -0.139, -0.133, -0.073, -0.054, -0.026, -0.021, -0.011, 0.112, 0.153, 0.169, 0.211, 0.298, 0.389, 0.395], 5: [-0.39, -0.368, -0.258, -0.208, -0.186, 0.012, 0.389, 0.501], 6: [-0.348, -0.258, -0.215, -0.139, -0.135, -0.106, -0.064, -0.015, 0.102, 0.129, 0.167, 0.173, 0.222, 0.322, 0.335, 0.385, 0.495], 7: [-0.433, -0.231, -0.151, -0.092, 0.056, 0.092, 0.187, 0.216, 0.433]}


# Make sequences and PREDICTION 

In [11]:
numb_seq = 1
for k,v in deltas.items():
    numb_seq *= len(deltas[k])
print(numb_seq/1000000,  'millions')

2.372112 millions


In [12]:
'''
data_train = pd.read_csv('train.csv')
X = data_train.drop(['Y'], axis='columns').values

ss = StandardScaler()
X = ss.fit_transform(X)
'''

"\ndata_train = pd.read_csv('train.csv')\nX = data_train.drop(['Y'], axis='columns').values\n\nss = StandardScaler()\nX = ss.fit_transform(X)\n"

In [0]:
mm = MinMaxScaler()

In [17]:
keys = list(deltas.keys())
ini_key = keys[0]
sequence = []
counter = 0
data_out = []

def tree_2(k, seq_ini, data_out, counter=counter):
    for v in deltas[k]:
        sequence = seq_ini.copy()
        sequence.append(v)
        
        if counter % 10000 == 0 and len(sequence) == 6:
                print('!',counter/10000)
                print(len(data_out))

        #if counter % 100000 == 0 and len(sequence) == 6:
                #print(sequence)
        if len(sequence) == len(deltas.items()):
            counter += 1
            sequence_np = np.array([sequence]).reshape(-1, 1)
            sequence_np = mm.fit_transform(sequence_np).reshape(1, -1)
            sequence_np = np.expand_dims(sequence_np, axis=2)
            
            prediction = model.predict(sequence_np)
            #print(prediction,sequence,sequence_np)
            #a=input()
            
            if prediction > 0.5:
                to_out = []
                to_out.append(prediction[0].tolist()[0])
                to_out.append(sequence)
                to_out.append(np.squeeze(sequence_np, axis=2).tolist()[0])
                data_out.append(to_out)
                #print(to_out)
                '''
                print('')
                for el in data_out:
                  print(el)
                a=input()
                '''
        else:
            counter, data_out = tree_2(k+1, sequence, data_out, counter=counter)
    return counter, data_out
        
counter, data_out = tree_2(ini_key,sequence, data_out, counter=counter)

! 0.0
0
! 1.0
0
! 2.0
0
! 3.0
0
! 4.0
0
! 5.0
14
! 6.0
29
! 7.0
29
! 8.0
58
! 9.0
58
! 10.0
72
! 11.0
86
! 12.0
86
! 13.0
116
! 14.0
116
! 15.0
140
! 16.0
142
! 17.0
142
! 18.0
142
! 19.0
142
! 20.0
142
! 21.0
142
! 22.0
142
! 23.0
142
! 24.0
142
! 25.0
142
! 26.0
142
! 27.0
142
! 28.0
142
! 29.0
142
! 30.0
142
! 31.0
142
! 32.0
142
! 33.0
142
! 34.0
142
! 35.0
142
! 36.0
142
! 37.0
142
! 38.0
142
! 39.0
142
! 40.0
142
! 41.0
142
! 42.0
142
! 43.0
142
! 44.0
142
! 45.0
142
! 46.0
142
! 47.0
145
! 48.0
145
! 49.0
145
! 50.0
146
! 51.0
146
! 52.0
170
! 53.0
170
! 54.0
170
! 55.0
198
! 56.0
198
! 57.0
211
! 58.0
211
! 59.0
211
! 60.0
226
! 61.0
226
! 62.0
226
! 63.0
226
! 64.0
226
! 65.0
226
! 66.0
226
! 67.0
226
! 68.0
226
! 69.0
226
! 70.0
226
! 71.0
226
! 72.0
226
! 73.0
226
! 74.0
226
! 75.0
226
! 76.0
226
! 77.0
226
! 78.0
226
! 79.0
226
! 80.0
226
! 81.0
226
! 82.0
226
! 83.0
226
! 84.0
226
! 85.0
226
! 86.0
226
! 87.0
226
! 88.0
226
! 89.0
226
! 90.0
226
! 91.0
226
! 92.0
226
! 93.

In [19]:
#out.txt
from google.colab import files

uploaded = files.upload()


In [22]:
for line in data_out:
  print(line)

[0.9779029488563538, [-0.465, -0.354, -0.237, -0.186, -0.064, 0.056], [0.0, 0.21305182341650675, 0.4376199616122841, 0.5355086372360844, 0.7696737044145874, 1.0]]
[0.9713166356086731, [-0.465, -0.354, -0.237, -0.186, -0.064, 0.092], [0.0, 0.19928186714542195, 0.4093357271095153, 0.5008976660682227, 0.7199281867145422, 1.0]]
[0.7914602756500244, [-0.465, -0.354, -0.237, -0.186, -0.064, 0.187], [0.0, 0.17024539877300626, 0.3496932515337424, 0.42791411042944794, 0.6150306748466259, 1.0]]
[0.6607378125190735, [-0.465, -0.354, -0.237, -0.186, -0.064, 0.216], [0.0, 0.16299559471365643, 0.3348017621145375, 0.40969162995594716, 0.5888399412628488, 1.0]]
[0.5095903277397156, [-0.465, -0.354, -0.237, -0.186, -0.015, 0.187], [0.0, 0.17024539877300626, 0.3496932515337424, 0.42791411042944794, 0.6901840490797547, 1.0]]
[0.652863085269928, [-0.465, -0.354, -0.237, 0.012, 0.222, 0.433], [0.0, 0.12360801781737196, 0.2538975501113586, 0.5311804008908686, 0.765033407572383, 1.0]]
[0.627185046672821, [-0

In [0]:
file_out = open('out.txt', 'w')

for line in data_out:
  el1 = ', '.join(list(map(str,line[1])))
  el2 = ', '.join(list(map(str,line[2]))) 
  el0 = str(line[0])
  file_out.write(' ,'.join([el0, el1, el2]) + '\n')
  
file_out.close()

In [0]:
files.download('out.txt')